<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using OpenAI Embeddings Search with SingleStoreDB</h1>
    </div>
</div>

## Setup

In [4]:
!pip cache purge --quiet

In [5]:
!pip install openai --quiet
!pip install tabulate --quiet

In [7]:
import getpass
import numpy as np
import openai
import os
import pandas as pd
import shutil
import wget
import zipfile

from ast import literal_eval
from openai import OpenAI
from tabulate import tabulate

In [8]:
EMBEDDING_MODEL = "text-embedding-ada-002"

In [9]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()

OpenAI API Key: ········


## Load Data

In [10]:
embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

'vector_database_wikipedia_articles_embedded.zip'

In [11]:
zip_file_name = "vector_database_wikipedia_articles_embedded.zip"
csv_file_name = "vector_database_wikipedia_articles_embedded.csv"
directory_name = "extract"

with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
    zip_ref.extractall(directory_name)

In [13]:
article_df = pd.read_csv(directory_name + "/" + csv_file_name)

In [14]:
article_df.head()

,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


In [15]:
# Apply literal_eval and cast to float32 for both columns in one step
article_df[["title_vector", "content_vector"]] = article_df[["title_vector", "content_vector"]].applymap(
    lambda x: np.array(literal_eval(x), dtype = np.float32)
)

# Set vector_id to be a string
article_df["vector_id"] = article_df["vector_id"].apply(str)

In [27]:
article_df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   url             25000 non-null  object
 2   title           25000 non-null  object
 3   text            25000 non-null  object
 4   title_vector    25000 non-null  object
 5   content_vector  25000 non-null  object
 6   vector_id       25000 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


## SingleStoreDB

### Create Table

In [30]:
%%sql
CREATE DATABASE IF NOT EXISTS openai_demo;

USE openai_demo;

DROP TABLE IF EXISTS wikipedia;
CREATE TABLE IF NOT EXISTS wikipedia (
    id INT PRIMARY KEY,
    url VARCHAR(255),
    title VARCHAR(100),
    text TEXT,
    title_vector VECTOR(1536),
    content_vector VECTOR(1536),
    vector_id INT
)

1 rows affected.

++
||
++
++

### Populate Table

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [32]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [33]:
article_df.to_sql(
    "wikipedia",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

25000

### Search Data

In [35]:
def search_wikipedia(
    query: str,
    column1: str,
    column2: str,
    num_rows: int = 10
) -> list:
    """Searches Wikipedia for the given query and returns the top `num_rows` results.

    Args:
        query: The query to search for.
        column1: The name of the column in the Wikipedia database to return for each result.
        column2: The name of the column in the Wikipedia database to use as the score for each result.
        num_rows: The number of results to return.

    Returns:
        A list of the top `num_rows` results.
    """

    # Get the embedding of the query
    query_embedding_response = client.embeddings.create(
        input = query,
        model = EMBEDDING_MODEL
    )
    query_embedding = query_embedding_response.data[0].embedding

    # Create the SQL statement
    stmt = """
        SELECT {column1}, (%s <*> {column2}) AS score
        FROM wikipedia
        ORDER BY score DESC
        LIMIT %s
    """.format(column1 = column1, column2 = column2)
    
    results = pd.read_sql(
        stmt,
        db_connection,
        params = (str(query_embedding), num_rows)
    )

    # Separate the results into two lists
    values = results.iloc[:, 0].tolist()
    scores = results.iloc[:, 1].tolist()

    # Return the results
    return values, scores

In [36]:
values1, scores1 = search_wikipedia(
    query = "modern art in Europe",
    column1 = "title",
    column2 = "title_vector",
    num_rows = 5
)

In [37]:
# Combine the values and scores lists into a list of tuples
# Each tuple contains a value and its corresponding score
table_data1 = list(zip(values1, scores1))

# Add a rank column to the table data
table_data1 = [(i + 1,) + data for i, data in enumerate(table_data1)]

# Create the table
table1 = tabulate(table_data1, headers = ["Rank", "Title", "Score"])

# Print the table
print(table1)

  Rank  Title                    Score
------  --------------------  --------
     1  Museum of Modern Art  0.87496
     2  Western Europe        0.867415
     3  Renaissance art       0.864026
     4  Pop art               0.860158
     5  Northern Europe       0.854661


In [38]:
values2, scores2 = search_wikipedia(
    query = "Famous battles in Scottish history",
    column1 = "text",
    column2 = "content_vector",
    num_rows = 5
)

In [39]:
# Combine the values and scores lists into a list of tuples
# Each tuple contains a value and its corresponding score
table_data2 = list(zip([value[:50] for value in values2], scores2))

# Add a rank column to the table data
table_data2 = [(i + 1,) + data for i, data in enumerate(table_data2)]

# Create the table
table2 = tabulate(table_data2, headers = ["Rank", "Text", "Score"])

# Print the table
print(table2)

  Rank  Text                                                   Score
------  --------------------------------------------------  --------
     1  The Battle of Bannockburn, fought on 23 and 24 Jun  0.869272
     2  The Wars of Scottish Independence were a series of  0.861397
     3  Events                                              0.852443
         January 1 – Charles II crowned King of
     4  The First War of Scottish Independence lasted from  0.849582
     5  Robert I of Scotland (11 July 1274 – 7 June 1329)   0.846126


## Cleanup

In [40]:
os.remove(zip_file_name)

In [41]:
shutil.rmtree(directory_name)

In [42]:
%%sql
DROP TABLE IF EXISTS wikipedia;

++
||
++
++

In [43]:
%%sql
DROP DATABASE IF EXISTS openai_demo;

++
||
++
++

## License

Some code sections in this notebook were adapted from [Using Vector Databases for Embeddings Search](https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/Using_vector_databases_for_embeddings_search.ipynb) from OpenAI.

MIT License

Copyright (c) 2023 OpenAI

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.